# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298274380865                   -0.85    5.2
  2   -8.300192723444       -2.72       -1.25    1.1
  3   -8.300450882820       -3.59       -1.89    3.5
  4   -8.300462822631       -4.92       -2.75    1.6
  5   -8.300464226007       -5.85       -3.01    7.5
  6   -8.300464412667       -6.73       -3.18    1.0
  7   -8.300464524804       -6.95       -3.31    2.2
  8   -8.300464584487       -7.22       -3.45    1.1
  9   -8.300464629522       -7.35       -3.66    1.0
 10   -8.300464635206       -8.25       -3.76    1.0
 11   -8.300464641951       -8.17       -3.97    1.0
 12   -8.300464643930       -8.70       -4.46    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67434037591                   -0.70    7.2
  2   -16.67866950791       -2.36       -1.14    1.5
  3   -16.67921124083       -3.27       -1.86    4.0
  4   -16.67928183020       -4.15       -2.76    2.8
  5   -16.67928612286       -5.37       -3.19    5.4
  6   -16.67928621290       -7.05       -3.48    4.0
  7   -16.67928622158       -8.06       -3.95    1.0
  8   -16.67928622413       -8.59       -4.57    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32625653467                   -0.56    6.4
  2   -33.33413975551       -2.10       -1.00    1.4
  3   -33.33598687834       -2.73       -1.73    6.6
  4   -33.33616324434       -3.75       -2.64    5.1
  5   -33.33687899521       -3.15       -2.44    6.4
  6   -33.33691261285       -4.47       -2.60    3.8
  7   -33.33694383865       -4.51       -3.81    2.9
  8   -33.33694390138       -7.20       -4.06    5.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298427884676                   -0.85    5.2
  2   -8.300263105040       -2.74       -1.59    1.0
  3   -8.300460275736       -3.71       -2.66    3.9
  4   -8.300435863424   +   -4.61       -2.53    9.9
  5   -8.300464517272       -4.54       -3.72    1.9
  6   -8.300464625708       -6.96       -4.17    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32613985143                   -0.56    7.2
  2   -33.32445860087   +   -2.77       -1.27    1.0
  3   -21.94628671062   +    1.06       -0.54    8.0
  4   -33.15852961946        1.05       -1.37    7.6
  5   -33.26410744580       -0.98       -1.39    4.1
  6   -32.10859001286   +    0.06       -1.02    4.8
  7   -33.32695605793        0.09       -2.03    5.0
  8   -33.33407434522       -2.15       -2.05    2.8
  9   -33.33523974939       -2.93       -2.40    2.0
 10   -33.33663871343       -2.85       -2.68    2.4
 11   -33.33683893616       -3.70       -2.99    2.6
 12   -33.33694090425       -3.99       -3.51    3.6
 13   -33.33694277963       -5.73       -3.66    3.5
 14   -33.33694348146       -6.15       -3.81    2.6
 15   -33.33694302890   +   -6.34       -4.04    2.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.